In [11]:
from datasets import load_dataset
import os
import csv

# Create folder
save_dir = "my_images"
os.makedirs(save_dir, exist_ok=True)

# CSV file to store metadata
csv_path = os.path.join(save_dir, "metadata.csv")

# Create CSV header
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["index", "filename", "label", "model", "prompt"])

# Load streaming dataset
ds = load_dataset(
    "ComplexDataLab/OpenFake",
    split="train",
    streaming=True
)

# ------------- CONFIG -------------
target_real = 5000     # عدد الصور الحقيقية المطلوب
target_fake = 5000     # عدد الصور المزيفة المطلوب
# -----------------------------------

real_count = 0
fake_count = 0
index = 0

for example in ds:
    label = example["label"]

    # Ignore labels after reaching target
    if label == "real" and real_count >= target_real:
        continue
    if label == "fake" and fake_count >= target_fake:
        continue

    # Now process the image
    img = example["image"]

    # Fix JPEG saving issue
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")


    model = example["model"]
    prompt = example["prompt"]

    filename = f"{index:03d}_{label}.jpg"
    filepath = os.path.join(save_dir, filename)

    img.save(filepath)

    # Save metadata
    with open(csv_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([index, filename, label, model, prompt])

    # Update counters
    if label == "real":
        real_count += 1
    else:
        fake_count += 1

    index += 1

    # Stop when balanced
    if real_count >= target_real and fake_count >= target_fake:
        break

# -----------------------------
# Folder Size Calculation
# -----------------------------
def folder_size(path):
    total = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            total += os.path.getsize(fp)
    return total

size_bytes = folder_size(save_dir)
size_mb = size_bytes / (1024 * 1024)
size_gb = size_bytes / (1024 * 1024 * 1024)

print(f"\n📦 Folder size: {size_mb:.2f} MB ({size_gb:.4f} GB)")
print(f"Real images: {real_count}, Fake images: {fake_count}")
print("Done!")


Resolving data files:   0%|          | 0/526 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/526 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



📦 Folder size: 850.48 MB (0.8305 GB)
Real images: 5000, Fake images: 5000
Done!
